## Leyendo datos de TSPData

Primero vamos a coger una distribución de ciudades reales, en particular de Argentina haciendo uso del fichero del campus virtual.

In [101]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [102]:
import pandas as pd


In [103]:
from StringIO import StringIO

df = pd.read_table('ar9152.tsp', sep=' ', skiprows=7, skipfooter=1, index_col=0, names=['x','y'], engine='python')

Mostramos los datos para ver que el fichero se ha cargado correctamente

In [104]:
df.head()

,x,y
1,36266.6667,62550.0000
2,34600.0000,58633.3333
3,51650.0000,72300.0000
4,37800.0000,67683.3333
5,35428.0556,60174.1667


In [105]:
print df.tail()
# Borro el último para evitar el EOF del fichero
df = df[:-1]
print df.tail()

               x           y
9148  34411.1111  58798.3333
9149  34813.3333  58475.2778
9150  34878.0556  58135.8333
9151  34917.2222  58386.9444
9152  34976.9444  58686.9444
               x           y
9147  34383.0556  58848.8889
9148  34411.1111  58798.3333
9149  34813.3333  58475.2778
9150  34878.0556  58135.8333
9151  34917.2222  58386.9444


Ahora podemos leer las posiciones

In [106]:
posiciones=[(float(x),float(y)) for x, y in zip(df['x'],df['y'])]

Normalizamos y establecemos el límite de ciudades a 1000

In [107]:
pos = np.asarray(posiciones)
max_pos = pos.max(0)
min_pos = pos.min(0)
ratio_pos = max_pos - min_pos
pos = (pos-min_pos)/ratio_pos
num_ciudades = 1000
posiciones = map(tuple, pos[:num_ciudades]) #Me quedo con los primeros num_ciudades elementos


En este caso vamos a generar la matriz de costes a partir de la distancia euclidea

In [108]:
from scipy.spatial.distance import euclidean as distance

def get_matrix_costes(posiciones):
    num = len(posiciones)
    costes = np.array([distance(posiciones[c1],posiciones[c2]) for c1 in range(num) for c2 in range(num)])
    return costes.reshape((num, num))

In [109]:
costes = get_matrix_costes(posiciones)

Costes ahora almacena para cada par de ciudades su distancia. Presentamos los datos por pantalla para asegurarnos de que funciona correctamente.

In [110]:
print costes

[[ 0.          0.21083606  0.69047883 ...,  0.32414658  0.2792496
   0.29056059]
 [ 0.21083606  0.          0.88140745 ...,  0.37827615  0.4061588
   0.39974159]
 [ 0.69047883  0.88140745  0.         ...,  0.90236424  0.77748313
   0.81120306]
 ..., 
 [ 0.32414658  0.37827615  0.90236424 ...,  0.          0.13229269
   0.09829191]
 [ 0.2792496   0.4061588   0.77748313 ...,  0.13229269  0.          0.03449672]
 [ 0.29056059  0.39974159  0.81120306 ...,  0.09829191  0.03449672  0.        ]]


# Función de fitness


La función de fitness consiste en calcular la distancia recorrida.

In [111]:
def fitness(solucion, costes):
    sum = 0
    
    for i in range(num_ciudades-1):
        c1 = solucion[i]
        c2 = solucion[i+1]
        sum += costes[c1,c2]
    # Volver a la ciudad inicial
    ciudad_final = solucion[num_ciudades-1]
    ciudad_origen = solucion[0]
    sum += costes[ciudad_final, ciudad_origen]
    return sum

# Implementación de la Busqueda Local

En este caso no es de aplicación la búsqueda local

In [112]:
import numpy as np

In [113]:
def local_search(resultado, p1, p2):
    return np.concatenate((resultado[:p1], resultado[p1:p2+1][::-1], resultado[p2+1:]))

In [114]:
# Prueba unitaria
a = [0, 1, 2, 3, 6, 7, 8 ,9 ,10]
print local_search(a, 3, 6)


[ 0  1  2  8  7  6  3  9 10]


# Solución Aleatoria

Incluimos la solución aleatoria para poder realizar la comparativa

In [115]:
def random_solution(num_ciudades): # Para la comparativa
    return np.random.permutation(num_ciudades)

# Construcción Greedy Aleatorio

In [116]:
import random

def ramdom_greedy_construction(costes, lr):
    
    pendientes = range(0, len(costes))
    solucion = [] # S = {}
    
    while pendientes != [] :
        
        # Repetir Mientras (no se haya construido la solución)
        # Si nos queda más de un elemento tenemos que decidir
        if 1 != len(pendientes):
            
            # Extraemos el primero que será parte de la solución
            ciudad = pendientes[0]
            solucion += [ciudad]

            # Obtenemos los costes para la ciudad seleccionada
            distancias = costes[ciudad]

            # Borramos la seleccionada de la lista de pendientes
            del pendientes[0]

            # Nos quedamos sólo con los costes de las ciudades pendientes
            # No vamos a volver a pasar por una ciudad
            distancias_destino = np.array([distancias[i] for i in pendientes])

            # Crear Lista Restringida de Candidatos (LRC)
            # Nos tenemos que quedar con las tres (lr) minimas
            # y seleccionar una al azar
                     
            if lr < len(distancias_destino):

                lr_minimos = distancias_destino.argsort()[:lr]                 
                index_seleccionado = lr_minimos[randint(0,lr)]    # s: selección-aleatoria-elemento
                    
            else:
                # Cuando no hay un conjunto de datos mínimo para aplicar el algoritmo 
                # nos quedamos con el mínimo. En la clase vimos que debía ser aleatorio
                # hacemos este aporte como una pequeña mejora al algoritmo.
                index_destino_mas_cercano_pendientes = distancias_destino.argmin()
                index_seleccionado = index_destino_mas_cercano_pendientes

            # Ponemos la siguiente ciudad como primera para que sea
            # por la que continuamos para construir el camino
            pendientes[0], pendientes[index_seleccionado] = pendientes[index_seleccionado], pendientes[0]

        else:
            # Caso particular para cuando sólo queda un elemento
            solucion += [pendientes[0]]
            del pendientes[0]
 
    
    return solucion
    


# Procedimiento GRASP
Aplicacimos el procedimiento GRASP tal como se recoge en las transparencias de clase


In [117]:
from sys import maxint

def grasp_procedure(fitness, costes, evaluations):
    
    #Construimos una primera solución que sirva de partida
    lr = 3
    best_sol = ramdom_greedy_construction(costes, lr)
    best = fitness(best_sol, costes)
    
    # Repetimos hasta complir con el criterio de parada
    for r in range(evaluations-1):
        solucion = ramdom_greedy_construction(costes, lr)
        fitness_sol = fitness(solucion, costes)
        
        # Si la solución es mejor que la obtenida previamente
        # la guardamos como la mejor
        if (fitness_sol < best):
            best = fitness_sol
            best_sol = solucion
            
    return best_sol, best

# Definición del experimento
Al no ser determinístico tendremos que repetir el algoritmo GRASP 10 veces. Obtendremos la media para comparar el resultado con la propuesta aleatoria ejecutada también 10 veces.

In [121]:
evaluaciones_grasp = 5000;
repeticiones = 10;

# Ejecutamos repeticiones veces el algoritmo grasp y el aleatorio
parciales_grasp = 0
parciales_aleatorio = 0

for i in range(0, repeticiones):
    
    solucion, resultado = grasp_procedure(fitness, costes, evaluaciones_grasp)  
    parciales_grasp += resultado
    
    solucion_aleatoria = random_solution(num_ciudades)
    resultado_aleatorio = fitness(solucion_aleatoria, costes)
    parciales_aleatorio += resultado_aleatorio

# Calculamos la media
distancia_grasp = parciales_grasp / repeticiones;
distancia_aleatoria = parciales_aleatorio / repeticiones;

print 'RESULTADO MEDIO USANDO ALGORITMO GRASP:'
print distancia_grasp
print len(solucion)

# Comprobamos que no hay elementos repetidos en el resultado
# esto nos garantiza, junto a que la solución tiene
# 1000 elementos que se han recorrido todas las ciudades
seen = set()
uniq = [x for x in solucion if x not in seen and not seen.add(x)]  
print len(uniq)

print 'RESULTADO MEDIO USANDO ALGORITMO ALEATORIO:'
print distancia_aleatoria
print len(solucion_aleatoria)

print 'ÚLTIMA SOLUCIÓN GRASP'
print solucion


RESULTADO MEDIO USANDO ALGORITMO GRASP:
25.2907431305
1000
1000
RESULTADO MEDIO USANDO ALGORITMO ALEATORIO:
297.121363026
1000
ÚLTIMA SOLUCIÓN GRASP
[0, 31, 572, 718, 828, 751, 747, 688, 527, 58, 376, 867, 827, 804, 803, 696, 752, 190, 189, 545, 746, 893, 848, 128, 909, 220, 135, 7, 543, 224, 116, 269, 838, 438, 419, 82, 199, 424, 760, 793, 794, 169, 379, 451, 214, 693, 168, 695, 450, 217, 959, 775, 845, 389, 357, 12, 233, 505, 630, 263, 455, 456, 589, 475, 432, 342, 470, 924, 373, 925, 344, 202, 71, 481, 463, 70, 314, 185, 497, 368, 414, 222, 433, 437, 965, 110, 163, 858, 921, 434, 315, 410, 855, 327, 854, 111, 324, 411, 435, 441, 759, 758, 905, 963, 779, 685, 418, 744, 791, 165, 480, 109, 754, 385, 952, 420, 602, 251, 786, 639, 969, 774, 888, 461, 780, 951, 763, 593, 341, 454, 198, 18, 426, 692, 333, 334, 915, 515, 638, 254, 210, 733, 26, 288, 408, 679, 286, 966, 35, 869, 348, 297, 404, 268, 137, 266, 734, 673, 24, 710, 731, 641, 662, 138, 216, 730, 661, 175, 188, 660, 483, 612, 611,

# Resultados
Vemos como la solución haciendo uso del algortimo GRASP es mucho mejor que la solución aleatoria. El realizar 5000 evaluaciones hace que sea bastante más lento que el aleatorio -como era de esperar- pero es cierto que aún con valores de evaluaciones grasp mínimas (5) se observan resultados mucho mejores.